<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/AskAPatient/Experiment%20(Change%20testing%20dataset)/Codes/ask_pubmed_tweet_overlap_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
if os.path.exists('medical-concept-normalization'):
  print('folder exist')
else:
  os.mkdir(base_dir)

In [0]:
if os.path.exists('medical-concept-normalization/data_collection'):
  print('folder exist')
else:
  os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# datasets

train_csv = 'https://raw.githubusercontent.com/unt-iialab/medical-concept-normalization/master/AskAPatient/Experiment%20(Change%20testing%20dataset)/Training%20set/AskAPatient.fold-0.train.csv'
valid_csv = 'https://raw.githubusercontent.com/unt-iialab/medical-concept-normalization/master/AskAPatient/Experiment%20(Change%20testing%20dataset)/Validation%20set/AskAPatient.fold-0.validation.csv'
test_csv = 'https://raw.githubusercontent.com/unt-iialab/medical-concept-normalization/master/AskAPatient/Experiment%20(Change%20testing%20dataset)/Testing%20set/AskAPatient_overlap_10.csv'
fine_tuning_data = 'fine_tuning_language_model_pubmed_bigtweet_mixed.csv'

In [10]:
# training data set
train = pd.read_csv(train_csv, header=None)
print(train.head())

# validating data set
valid = pd.read_csv(valid_csv,header=None)
# valid.head()

# testing data set
test = pd.read_csv(test_csv,header=None)
# test.head()

   0                1
0  0             Fear
1  0             Fear
2  0             Fear
3  0  scared to death
4  1   Abdominal rash


In [12]:
# Creating DataBunch for language modelling
data_lm = TextLMDataBunch.from_csv(path, fine_tuning_data)

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [14]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas.add_test(test,label=0)

In [0]:
data_lm.save()
data_clas.save()

In [0]:
data_lm.train_ds.vocab.itos

In [18]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [19]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [20]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.184810,4.619129,0.250707,23:35


In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.443312,4.272274,0.284263,23:15


In [22]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.115612,4.233424,0.287825,23:09


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.069959,4.202822,0.290978,23:20


In [0]:
learn.save_encoder('tuning_pubmed_bigtweet_overlap_0_first')

In [25]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('tuning_pubmed_bigtweet_overlap_0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.790872,3.472463,0.453254,02:49


In [26]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.732496,2.998827,0.526627,02:45


In [27]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.297324,2.779397,0.556213,03:24


In [28]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.043669,2.520519,0.586982,03:40
1,1.867559,2.422603,0.601183,03:29


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.847514,2.265691,0.621302,03:48
1,1.638289,2.191489,0.630769,03:40


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.485657,2.129978,0.637870,03:34
1,1.416922,2.065023,0.650888,03:44


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.410282,2.028491,0.650888,03:27
1,1.307991,1.969013,0.659172,03:27


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.275898,1.989940,0.654438,03:17
1,1.200837,1.951936,0.657988,03:36


In [33]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.142226,1.957662,0.656805,03:31
1,1.167240,1.901839,0.660355,03:42
2,1.071278,1.869027,0.666272,03:26
3,1.042827,1.847876,0.668639,03:24


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.013414,1.846162,0.675740,03:20
1,1.079856,1.915731,0.671006,03:23
2,0.999422,1.845222,0.687574,03:29
3,0.912758,1.835921,0.689941,03:39


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.951967,1.858126,0.671006,03:35
1,0.959408,1.801356,0.675740,03:41
2,0.982441,1.871876,0.694675,03:17
3,0.862528,1.802813,0.686391,03:36


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.874173,1.883415,0.686391,03:31
1,0.943756,1.927803,0.680473,03:28
2,0.925469,1.834317,0.689941,03:26
3,0.800072,1.840507,0.687574,03:38


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.846578,1.859841,0.685207,03:36
1,0.885689,1.922218,0.681657,03:20
2,0.843594,1.899104,0.692308,03:28
3,0.793595,1.894781,0.689941,03:30


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.790491,1.894112,0.685207,03:34
1,0.868103,1.964257,0.687574,03:34
2,0.784395,1.886315,0.684024,03:46
3,0.758694,1.879155,0.693491,03:42


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.820332,1.895893,0.693491,03:31
1,0.874494,2.008154,0.669823,03:41
2,0.799944,1.926943,0.689941,03:38
3,0.795067,1.921049,0.686391,03:46


In [40]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.838059,2.039027,0.680473,03:31
1,0.799161,1.915185,0.689941,03:32


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.802463,1.941115,0.684024,03:23
1,0.806167,1.973878,0.686391,03:31


In [42]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.769528,1.881295,0.693491,03:33


In [0]:
# save the best model

learn.save_encoder('tuning_pubmed_bigtweet_overlap_0_last')

# Part three: Predict on the test dataset

In [44]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [45]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

681
681
[840, 228, 122, 544, 5, 1026, 772, 9, 10, 726, 438, 71, 210, 27, 597, 877, 491, 76, 211, 30, 32, 175, 37, 37, 218, 348, 289, 269, 384, 384, 384, 49, 49, 245, 209, 729, 71, 71, 289, 76, 260, 190, 710, 190, 260, 260, 260, 260, 260, 211, 289, 260, 710, 80, 847, 89, 627, 377, 517, 742, 318, 826, 610, 746, 492, 98, 794, 644, 100, 100, 100, 100, 591, 766, 291, 214, 107, 108, 660, 209, 189, 313, 111, 189, 111, 111, 211, 111, 111, 548, 120, 1032, 710, 125, 710, 125, 127, 853, 129, 129, 289, 745, 809, 142, 144, 706, 415, 70, 331, 157, 189, 178, 180, 168, 646, 190, 742, 182, 260, 182, 289, 245, 183, 183, 187, 710, 710, 190, 190, 190, 190, 190, 190, 190, 190, 197, 34, 214, 214, 214, 135, 245, 96, 470, 210, 210, 210, 210, 289, 579, 710, 419, 214, 245, 214, 265, 710, 710, 553, 44, 604, 228, 79, 228, 289, 289, 228, 544, 746, 98, 235, 236, 265, 386, 710, 710, 470, 187, 187, 710, 710, 710, 710, 710, 728, 318, 245, 471, 470, 173, 245, 846, 246, 298, 710, 710, 728, 259, 289, 289, 710, 265, 419, 

In [46]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

259
0.3803230543318649
